In [2]:
import Funtions_mod
import pandas as pd
import numpy as np
from pathlib import Path
import numpy as np
from classLib_forFlightplans import *
from shapely.geometry import Polygon, Point
from shapely.ops import transform
from functools import partial
import pyproj
import random
import operator
from functools import reduce
import geopandas as gpd

In [3]:
Path.cwd()

WindowsPath('e:/ATMRI/CD_R/ATCO_preferences_interface/Scenario_interface_work/python-code')

In [4]:
region = [100,0,110,0,10,110,10,100]
d1 = Funtions_mod.airwayTransofrmation_xy(region)
df = pd.DataFrame(d1)
print('datatype of transformed airways file=' ,type(d1))

sectors = Funtions_mod.sectorTransformation(region)
print('datatype of transformed sectors file=' , type(sectors))

datatype of transformed airways file= <class 'list'>
datatype of transformed sectors file= <class 'dict'>


In [5]:
datalist=[]
for i in range(15):
    waypointList = []
    for i in range(len(df)):
        for j in range(len(df.segments[i])):
            waypointList.append(WayPoint(df.segments[i][j]['name'],[df.segments[i][j]['transf_x'],900-df.segments[i][j]['transf_y']]))
            
    len(waypointList),len(df.segments)

    airwayList=[]
    a=0
    for i in range(len(df.segments)):
        x=len(df.segments[i])
        # print(x)
        for j in range(a,len(waypointList[:a+x])-1):
            li=[waypointList[j],waypointList[j+1]]
            # print(li)
            airwayList.append(Airways('aw'+ str(j),li[0],li[1]))
        a+=x

    route1 = ['MELAS (VVTS/WSJC FIR BDRY)','MABLI','MUMSO','MABAL','KILOT','KIBOL WSJC/WMFC FIR BDRY','PEKLA']
    route2 = ['ENREP','NOPAT','DAMOG','SUSAR','MUMSO','MABAL','KILOT','KIBOL WSJC/WMFC FIR BDRY','PEKLA']


    r1=[]
    for i in (route1):
        for j in range(len(waypointList)):
            if i == waypointList[j].name:
                r1.append(waypointList[j])
                break
    r2=[]
    for i in (route2):
        for j in range(len(waypointList)):
            if i == waypointList[j].name:
                r2.append(waypointList[j])
                break

    ac_paths=[r1,r2]
    aircraft_type=['medium','heavy','super_heavy']

    ac_per_scenario = 2 
    aircraftList=[]
    aircraft_name=[]
    routelist=[]
    for n in range(1,ac_per_scenario+1):
        aircraft_name.append('ac'+str(n))
    for i in range(ac_per_scenario):
        route = np.random.choice(ac_paths)
        
        d=[10,20,30,40,50,60,65,70,75,80,85,90,95,100,110,120] #offset,
        aircraft = Aircraft(str(aircraft_name[i]),route[0],route[1], route,np.random.choice(d)) #create multiple scenarios
        aircraftList.append(aircraft)
        routelist.append(route)

    aircraft_paths_x = []
    aircraft_paths_y = []
    path_waypoints   = []
    for i in range(len(routelist)):
        x_list=[]
        y_list=[]
        wps_list=[]
        for j in range(len((routelist[i]))):
            wps_list.append(routelist[i][j].name)
            x=routelist[i][j].x
            y=routelist[i][j].y
            x_list.append(x)
            y_list.append(y)
        aircraft_paths_x.append(x_list)
        aircraft_paths_y.append(y_list)
        path_waypoints.append(wps_list)


    x_dirlist=[]
    y_dirlist=[]
    for i in range(len(aircraftList)):
        x_dir =[]
        y_dir =[]
        for j in range(len(aircraftList[i].route)-1):
            startpoint = np.array([aircraftList[i].route[j].x, aircraftList[i].route[j].y])
            endpoint = np.array([aircraftList[i].route[j+1].x, aircraftList[i].route[j+1].y])
            segment = np.linalg.norm(startpoint - endpoint)
            xdir = (endpoint[0] - startpoint[0])/segment
            ydir = (endpoint[1] - startpoint[1])/segment
            x_dir.append(xdir)
            y_dir.append(ydir)
        x_dirlist.append(x_dir)
        y_dirlist.append(y_dir)


    vel=0.25
    timelist=[]
    x_loclist=[]
    y_loclist=[]
    total_time=[]
    for i in range(len(aircraftList)):
        x_loc=[]
        y_loc=[]
        time=[]
        section_time=[]
        for j in range(len(aircraftList[i].route)-1):
            startpoint = np.array([aircraftList[i].route[j].x, aircraftList[i].route[j].y])
            endpoint = np.array([aircraftList[i].route[j+1].x, aircraftList[i].route[j+1].y])
            distance = np.linalg.norm(startpoint - endpoint)
            totaltime = distance/vel    #(time per kilometer in seconds, bwtween two waypoints)
            
            for t in range(int(totaltime)):
                x_point = startpoint[0] + t*x_dirlist[i][j]*vel
                y_point = startpoint[1] + t*y_dirlist[i][j]*vel
                x_loc.append(x_point)
                y_loc.append(y_point)
                time.append(t)
                
            section_time.append(totaltime)
        # print(len(section_time))
        timelist.append(time)
        x_loclist.append(x_loc)
        y_loclist.append(y_loc)
        total_time.append(section_time)

        
    #repeating the directins vectors based on total time
    all_dirs_x=[]
    all_dirs_y=[]
    for i in range(len(total_time)): #2
        x_list=[]
        y_list=[]
        for j in range(len(total_time[i])):
            x = np.repeat(x_dirlist[i][j], int(total_time[i][j]))
            y = np.repeat(y_dirlist[i], int(total_time[i][j]))
            # print(len(x))
            x= x.tolist()
            y= y.tolist()
            # print(len(x))
            # print(len(y[]))
            x_list.append(x)
            y_list.append(y)
        
        all_dirs_x.append(x_list)
        all_dirs_y.append(y_list)

    x_directions_flat=[]
    y_directions_flat=[]
    for i in range(len(x_dirlist)): #2
        x_l= all_dirs_x[i]
        x_flat = reduce(operator.concat,x_l)
        # print(x_flat)
        x_directions_flat.append(x_flat)

        y_l = all_dirs_y[i]
        y_flat = reduce(operator.concat,y_l)
        y_directions_flat.append(y_flat)

    # len(x_directions_flat[0])

    dict4={}
    for i in range(len(waypointList)):
        waydict={waypointList[i].name:{
            'x':waypointList[i].x,
            'y':waypointList[i].y}}
        dict4.update(waydict)


    dict5={}
    for i in range(len(airwayList)):
        airwaydict= {airwayList[i].name:{
            'start':airwayList[i].start_wp,
            'end' : airwayList[i].end_wp}}
        dict5.update(airwaydict)


    dict6={}
    for i in range(len(aircraftList)):
        aircraftdict= {
            aircraftList[i].name:{
            'x': aircraftList[i].offset_x,
            'y':aircraftList[i].offset_y,
            'dir_x': x_dirlist[i][0],
            'dir_y': y_dirlist[i][0],
            'x_dirs':x_directions_flat[i],
            'y_dirs':y_directions_flat[i],
            'type': np.random.choice(aircraft_type),
            'altitude':360,
            'path_waypoints': path_waypoints[i],
            'x_path' : x_loclist[i],
            'y_path' : y_loclist[i],
            'timestep': timelist[i]
            }
            }
        dict6.update(aircraftdict)

    dd={'waypoints':dict4, 'airways':dict5,'aircrafts':dict6}
    data_dict = {**dd,**sectors}
    datalist.append(data_dict)

import json
with open('scenario_data/28_Test_2.json', 'w') as json_file:
    json.dump(datalist, json_file)

In [13]:
dd


In [ ]:
asd={'a':1,'b':2}
